# Persian Stance Classification - Deep Learning

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import numpy as np
import numpy as np
import os.path as path

# Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# input files
cleaned_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/Clean_Claim_Body.csv"
train_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/train_data.csv"
test_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/test_data.csv"

FEATURES_DIR = "/content/drive/MyDrive/Stance Detection Project/Baseline Zarharan/features/"

feature_file_negated = FEATURES_DIR + "negated.npy"
feature_file_ngram = FEATURES_DIR + "ngrams.npy"
feature_file_hand = FEATURES_DIR + "hand.npy"
sequences_file = FEATURES_DIR + "sequences.npy"

# output files
deep_model_path = "/content/drive/MyDrive/persian_stance_deep_data/Baseline Zarharan/models/model_v1.h5"

# Read Cleaned Data from CSV File

In [ ]:
import pandas as pd
dataset_clean = pd.read_csv(cleaned_path, index_col = 0, )

In [ ]:
clean_claim = dataset_clean['claim']
clean_body = dataset_clean['body']

In [ ]:
dataset_clean.head()

,claim,body,label
0,کلاهبرداری از رانندگان با شگرد نشت بنزین !,به گزارش خبرنگار گروه جامعه خبرگزاری میزان،29 ...,Discuss
1,تجاوز به دختر بازداشت شده و واژگونی ون گشت ارش...,انتشار کلیپ واژگونی ماشین گشت ارشاد توسط مردم ...,Discuss
2,تعظیم 20 دقیقه ای وزیر نیرو ژاپن به علت قطع بر...,وزیر نیروی ژاپن به علت قطع شدن برق؛ به همان مد...,Agree
3,سرمربیگری گاس هیدینک برای تراکتورسازی,به تازگی محمد تقوی استعفای خود را از سرمربیگری...,Discuss
4,کشف موجود عجیبی شبیه انسان در یک حفاری در پاکس...,پس از 20 سال حفاری با دقتی باورنکردنی، سرانجام...,Unrelated


# Read Data Train and Test from CSV Files

In [ ]:
data_train = pd.read_csv(train_path, index_col = 0, )
data_test = pd.read_csv(test_path, index_col = 0, )

In [ ]:
len(data_train), len(data_test)

(1597, 400)

# Load Features

In [ ]:
x_negated = np.load(feature_file_negated)
x_hand = np.load(feature_file_hand)
x_ngram = np.load(feature_file_ngram)

X_feat = []
X_feat.append(x_negated)
X_feat.append(x_hand)
X_feat.append(x_ngram)

X_MLP = np.concatenate(X_feat, axis=1)
y_MLP = dataset_clean['label'].tolist()

In [ ]:
print(X_MLP.shape)

(1997, 20026)


In [ ]:
X_LSTM = np.load(sequences_file)
y_LSTM = dataset_clean['label'].tolist()

In [ ]:
X_LSTM.shape

(1997, 100)

In [ ]:
claims = dataset_clean['claim'].tolist()
bodies = dataset_clean['body'].tolist()

concatenated = []
for i in range(len(claims)):
    concatenated.append(claims[i] + ". " + bodies[i])

X_BERT = concatenated
y_BERT = dataset_clean['label'].tolist()

In [ ]:
#X_BERT[0:10]

# Deep Learning Model

In [ ]:
import numpy as np
import os.path as path
import pickle

from keras.layers.core import Dense
from keras.layers import LSTM
from keras import optimizers
from keras.models import Model, load_model
from keras.layers import concatenate
from keras.layers import Embedding, Input

In [ ]:
from sklearn.model_selection import train_test_split

X_train_MLP, X_test_MLP, y_train, y_test = train_test_split(
    X_MLP, y_MLP, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

X_train_LSTM, X_test_LSTM, y_train, y_test = train_test_split(
    X_LSTM, y_LSTM, test_size=0.2, random_state=42)

In [ ]:
def convert_data_to_one_hot(y_train):

    y_train_temp = np.zeros((y_train.size, y_train.max() + 1), dtype=np.int)
    y_train_temp[np.arange(y_train.size), y_train] = 1

    return y_train_temp

In [ ]:
from sklearn import preprocessing
import keras

le = preprocessing.LabelEncoder()
le.fit(y_train)
print(list(le.classes_))
y_train = le.transform(y_train)

le = preprocessing.LabelEncoder()
le.fit(y_test)
print(list(le.classes_))
y_test = le.transform(y_test)

print(y_test[0:10])

['Agree', 'Disagree', 'Discuss', 'Unrelated']
['Agree', 'Disagree', 'Discuss', 'Unrelated']
[2 2 1 3 2 0 1 2 3 2]


In [ ]:
def calculate_class_weight(y_train, no_classes=2):
    # https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras
    from sklearn.utils import class_weight

    class_weight_list = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {}
    for i in range(no_classes):
        class_weights[i] = class_weight_list[i]
    print(class_weights)
    return class_weights

In [ ]:
y_train_one_hot = convert_data_to_one_hot(y_train)
y_test_one_hot = convert_data_to_one_hot(y_test)

param_dict="single_flat_LSTM_50d_100"

PARAM_DICT_FILENAME = param_dict + "_param_dict.pkl"


# load feature dict for LSTM_1000_GloVe
with open(FEATURES_DIR+PARAM_DICT_FILENAME, "rb") as f:
  param_dict = pickle.load(f)

# load parameters needed for embedding layer
EMBEDDING_DIM = param_dict["EMBEDDING_DIM"] # e.g. 50
MAX_SEQ_LENGTH = param_dict["MAX_SEQ_LENGTH"] # e.g. 100

# load embeddings
EMBEDDING_FILE = np.load(FEATURES_DIR+param_dict["EMBEDDING_FILE"])

print("EMBEDDING_FILE.shape = " + str(EMBEDDING_FILE.shape))

# calc cass weights
class_weights = calculate_class_weight(y_train, no_classes=4)

EMBEDDING_FILE.shape = (48758, 50)
{0: 3.6295454545454544, 1: 2.4344512195121952, 2: 0.45733104238258876, 3: 0.8872222222222222}


<ipython-input-18-d1e925b257f1>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train_temp = np.zeros((y_train.size, y_train.max() + 1), dtype=np.int)


In [ ]:
kernel_initializer = 'glorot_uniform'
regularizer = None
batch_size=200
dense_activity_regularizer=None
LSTM_implementation = 2


################
# CLAIMS LAYER #
################
lstm_input = Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name='lstm_input') # receive sequences of MAX_SEQ_LENGTH_CLAIMS integers
embedding = Embedding(input_dim=len(EMBEDDING_FILE), # lookup table size
                                    output_dim=EMBEDDING_DIM, # output dim for each number in a sequence
                                    weights=[EMBEDDING_FILE],
                                    input_length=MAX_SEQ_LENGTH, # receive sequences of MAX_SEQ_LENGTH_CLAIMS integers
                                    mask_zero=True,
                                    trainable=True)(lstm_input)
data_LSTM = LSTM(
            100, return_sequences=True, stateful=False, dropout=0.2,
            batch_input_shape=(batch_size, MAX_SEQ_LENGTH, EMBEDDING_DIM),
            input_shape=(MAX_SEQ_LENGTH, EMBEDDING_DIM), implementation=LSTM_implementation
            )(embedding)
data_LSTM = LSTM(
            100, return_sequences=False, stateful=False, dropout=0.2,
            batch_input_shape=(batch_size, MAX_SEQ_LENGTH, EMBEDDING_DIM),
            input_shape=(MAX_SEQ_LENGTH, EMBEDDING_DIM), implementation=LSTM_implementation
            )(data_LSTM)

###############################
# MLP (NON-TIMESTEP) FEATURES #
###############################
mlp_input = Input(shape=(len(X_train_MLP[0]),), dtype='float32', name='mlp_input')

###############
# MERGE LAYER #
###############
merged = concatenate([data_LSTM, mlp_input])

dense_mid = Dense(600, kernel_regularizer=regularizer, kernel_initializer=kernel_initializer,
                          activity_regularizer=dense_activity_regularizer, activation='relu')(merged)
dense_mid = Dense(600, kernel_regularizer=regularizer, kernel_initializer=kernel_initializer,
                          activity_regularizer=dense_activity_regularizer, activation='relu')(dense_mid)
dense_mid = Dense(600, kernel_regularizer=regularizer, kernel_initializer=kernel_initializer,
                          activity_regularizer=dense_activity_regularizer, activation='relu')(dense_mid)
dense_out = Dense(4,activation='softmax', name='dense_out')(dense_mid)

# build model
model = Model(inputs=[lstm_input, mlp_input], outputs=[dense_out])

# print summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lstm_input (InputLayer)        [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 50)      2437900     ['lstm_input[0][0]']             
                                                                                                  
 lstm (LSTM)                    (None, 100, 100)     60400       ['embedding[0][0]']              
                                                                                                  
 lstm_1 (LSTM)                  (None, 100)          80400       ['lstm[0][0]']                   
                                                                                              

In [ ]:
LABELS = ['Agree', 'Disagree', 'Discuss', 'Unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm

In [ ]:
from keras.callbacks import Callback
import numpy as np
class EarlyStoppingOnF1(Callback):
    """
    Prints some metrics after each epoch in order to observe overfitting
                https://github.com/fchollet/keras/issues/5794
                custom metrics: https://github.com/fchollet/keras/issues/2607
    """

    def __init__(self, epochs,
                 X_test_claims,
                 X_test_orig_docs,
                 y_test, loss_filename, epsilon=0.0, min_epoch = 15, X_test_nt=None):
        self.epochs = epochs
        self.patience = 2
        self.counter = 0
        self.prev_score = 0
        self.epsilon = epsilon
        self.loss_filename = loss_filename
        self.min_epoch = min_epoch
        self.X_test_nt = X_test_nt
        #self.print_train_f1 = print_train_f1

        #self.X_train_claims = X_train_claims
        #self.X_train_orig_docs = X_train_orig_docs
        #self.X_train_evid = X_train_evid
        #self.y_train = y_train

        self.X_test_claims = X_test_claims
        self.X_test_orig_docs = X_test_orig_docs
        self.y_test = y_test
        Callback.__init__(self)

    def on_epoch_end(self, epoch, logs={}):
        if epoch + 1 < self.epochs:
            from sklearn.metrics import f1_score

            # get prediction and convert into list
            if type(self.X_test_orig_docs).__module__ == np.__name__ and type(self.X_test_nt).__module__ == np.__name__:
                predicted_one_hot = self.model.predict([
                    self.X_test_claims,
                    self.X_test_orig_docs,
                    self.X_test_nt
                ])
            elif type(self.X_test_orig_docs).__module__ == np.__name__:
                predicted_one_hot = self.model.predict([
                    self.X_test_claims,
                    self.X_test_orig_docs,
                ])
            else:
                predicted_one_hot = self.model.predict(self.X_test_claims)
            predict = np.argmax(predicted_one_hot, axis=-1)

            """
            predicted_one_hot_train = self.model.predict([self.X_train_claims, self.X_train_orig_docs, self.X_train_evid])
            predict_train = np.argmax(predicted_one_hot_train, axis=-1)


            # f1 for train data
            f1_macro_train = ""
            if self.print_train_f1 == True:
                f1_0_train = f1_score(self.y_train, predict_train, labels=[0], average=None)
                f1_1_train = f1_score(self.y_train, predict_train, labels=[1], average=None)
                f1_macro_train = (f1_0_train[0] + f1_1_train[0]) / 2
                print(" - train_f1_(macro): " + str(f1_macro_train))"""

            predicted = [LABELS[int(a)] for a in predict]
            actual = [LABELS[int(a)] for a in self.y_test]
            # calc FNC score
            fold_score, _ = score_submission(actual, predicted)
            max_fold_score, _ = score_submission(actual, actual)
            fnc_score = fold_score / max_fold_score
            print(" - fnc_score: " + str(fnc_score))

            # f1 for test data
            f1_0 = f1_score(self.y_test, predict, labels=[0], average=None)
            f1_1 = f1_score(self.y_test, predict, labels=[1], average=None)
            f1_2 = f1_score(self.y_test, predict, labels=[2], average=None)
            f1_3 = f1_score(self.y_test, predict, labels=[3], average=None)
            f1_macro = (f1_0[0] + f1_1[0] + f1_2[0] + f1_3[0]) / 4
            print(" - val_f1_(macro): " + str(f1_macro))
            print("\n")

            header = ""
            values = ""
            for key, value in logs.items():
                header = header + key + ";"
                values = values + str(value) + ";"
            if epoch == 0:
                values = "\n" + header + "val_f1_macro;" + "fnc_score;" + "\n" + values + str(f1_macro) + str(fnc_score) + ";"
            else:
                values += str(f1_macro) + ";" + str(fnc_score) + ";"
            append_to_loss_monitor_file(values, self.loss_filename)

            if epoch >= self.min_epoch-1:  # 9
                if f1_macro + self.epsilon <= self.prev_score:
                    self.counter += 1
                else:
                    self.counter = 0
                if self.counter >= 2:
                    self.model.stop_training = True
            #print("Counter at " + str(self.counter))
            self.prev_score = f1_macro
            #print("\n")

In [ ]:
def append_to_loss_monitor_file(text, filepath):
    with open(filepath, 'a+') as the_file:
        the_file.write(text+"\n")

In [ ]:
%%time

from tensorflow.keras import optimizers

lr = 0.001
optimizer_name = "RMS"
use_class_weights = True
epochs = 70
min_epoch = 20
save_folder = None
loss_filename = 'loss.h5'

# optimizers
if optimizer_name == "adagrad":
  optimizer = optimizers.Adagrad(lr=lr)
  print("Used optimizer: adagrad, lr="+str(lr))
elif optimizer_name == "adamax":
  optimizer = optimizers.Adamax(lr=lr)
  print("Used optimizer: adamax, lr="+str(lr))
elif optimizer_name == "nadam":
  optimizer = optimizers.Nadam(lr=lr)  # recommended to leave at default params
  print("Used optimizer: nadam, lr="+str(lr))
elif optimizer_name == "rms":
  optimizer = optimizers.RMSprop(lr=lr)  # recommended for RNNs
  print("Used optimizer: rms, lr="+str(lr))
elif optimizer_name == "SGD":
  optimizer = optimizers.SGD(lr=lr)  # recommended for RNNs
  print("Used optimizer: SGD, lr="+str(lr))
elif optimizer_name == "adadelta":
  optimizer = optimizers.Adadelta(lr)  # recommended for RNNs
  print("Used optimizer: adadelta, lr="+str(lr))
else:
  optimizer = optimizers.Adam(lr=lr)
  print("Used optimizer: Adam, lr=" + str(lr))

# compile model
model.compile(optimizer, 'kullback_leibler_divergence', # categorial_crossentropy
                           metrics=['accuracy'])
if use_class_weights == True:
  hist = model.fit([X_train_LSTM, X_train_MLP],
                             y_train_one_hot,
                             validation_data=([X_test_LSTM, X_test_MLP], y_test_one_hot),
                             batch_size=batch_size, epochs=epochs, verbose=1, class_weight=class_weights,
                              callbacks=[
                               EarlyStoppingOnF1(epochs,
                                                 X_test_LSTM, X_test_MLP, y_test,
                                                 loss_filename, epsilon=0.0, min_epoch=min_epoch),])
else:
  hist = model.fit([X_train_LSTM, X_train_MLP],
                             y_train_one_hot,
                             validation_data=([X_test_LSTM, X_test_MLP], y_test_one_hot),
                             batch_size=batch_size, epochs=epochs, verbose=1,
                           callbacks=[
                               EarlyStoppingOnF1(epochs,
                                                 X_test_LSTM, X_test_MLP, y_test,
                                                 loss_filename, epsilon=0.0, min_epoch=min_epoch),])



Used optimizer: Adam, lr=0.001
Epoch 1/70
13/13 [==============================] - 4s 14ms/step
 - fnc_score: 0.5901639344262295
 - val_f1_(macro): 0.41010925133555365


8/8 [==============================] - 32s 1s/step - loss: 1.3160 - accuracy: 0.3369 - val_loss: 1.1453 - val_accuracy: 0.5750
Epoch 2/70
13/13 [==============================] - 0s 10ms/step
 - fnc_score: 0.4139344262295082
 - val_f1_(macro): 0.3267886656162103


8/8 [==============================] - 2s 297ms/step - loss: 1.1080 - accuracy: 0.5642 - val_loss: 1.2043 - val_accuracy: 0.3350
Epoch 3/70
13/13 [==============================] - 0s 10ms/step
 - fnc_score: 0.4713114754098361
 - val_f1_(macro): 0.4058401504462841


8/8 [==============================] - 2s 292ms/step - loss: 0.7850 - accuracy: 0.6237 - val_loss: 1.1669 - val_accuracy: 0.3825
Epoch 4/70
13/13 [==============================] - 0s 10ms/step
 - fnc_score: 0.6789617486338798
 - val_f1_(macro): 0.58424138610627


8/8 [============================

In [ ]:
model.save("save.h5")

In [ ]:
model.save(deep_model_path)

In [ ]:
print("Loading model from:" + "save.h5")
model = load_model("save.h5")
predicted_one_hot = model.predict([X_test_LSTM, X_test_MLP])
predicted_int = np.argmax(predicted_one_hot, axis=-1)

Loading model from:save.h5
13/13 [==============================] - 4s 12ms/step


In [ ]:
y_test_str = le.inverse_transform(y_test)
y_pred_str = le.inverse_transform(predicted_int)

In [ ]:
# y_test_str

In [ ]:
# y_pred_str

In [ ]:
score, cm = score_submission(y_test_str, y_pred_str)
fold_score, _ = score_submission(y_test_str, y_pred_str)
max_fold_score, _ = score_submission(y_test_str, y_test_str)
score = fold_score / max_fold_score

print("FNC-1 score from restored model: " +  str(score) +"\n")

FNC-1 score from restored model: 0.7465846994535519



In [ ]:
fold_score

273.25

In [ ]:
max_fold_score

366.0

In [ ]:
def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


In [ ]:
print_confusion_matrix(cm)

-------------------------------------------------------------
|           |   Agree   | Disagree  |  Discuss  | Unrelated |
-------------------------------------------------------------
|   Agree   |     8     |     0     |    17     |     2     |
-------------------------------------------------------------
| Disagree  |     2     |    24     |    16     |     0     |
-------------------------------------------------------------
|  Discuss  |     9     |     5     |    163    |    18     |
-------------------------------------------------------------
| Unrelated |     1     |     2     |    45     |    88     |
-------------------------------------------------------------


In [ ]:
from sklearn.metrics import classification_report

print( classification_report(y_test_str, y_pred_str) )

# macro avg = unweighted mean of F1 values ***
# weighted avg = weighted mean of F1 values based on classes support/number of instances
# micro avg = a global F1 value based on sum of TPs, FPs, FNs

              precision    recall  f1-score   support

       Agree       0.40      0.30      0.34        27
    Disagree       0.77      0.57      0.66        42
     Discuss       0.68      0.84      0.75       195
   Unrelated       0.81      0.65      0.72       136

    accuracy                           0.71       400
   macro avg       0.67      0.59      0.62       400
weighted avg       0.72      0.71      0.70       400



# Results

In [ ]:
# Model: "model"
# __________________________________________________________________________________________________
#  Layer (type)                   Output Shape         Param #     Connected to
# ==================================================================================================
#  lstm_input (InputLayer)        [(None, 100)]        0           []

#  embedding (Embedding)          (None, 100, 50)      2437900     ['lstm_input[0][0]']

#  lstm (LSTM)                    (None, 100, 100)     60400       ['embedding[0][0]']

#  lstm_1 (LSTM)                  (None, 100)          80400       ['lstm[0][0]']

#  mlp_input (InputLayer)         [(None, 20026)]      0           []

#  concatenate (Concatenate)      (None, 20126)        0           ['lstm_1[0][0]',
#                                                                   'mlp_input[0][0]']

#  dense (Dense)                  (None, 600)          12076200    ['concatenate[0][0]']

#  dense_1 (Dense)                (None, 600)          360600      ['dense[0][0]']

#  dense_2 (Dense)                (None, 600)          360600      ['dense_1[0][0]']

#  dense_out (Dense)              (None, 4)            2404        ['dense_2[0][0]']

# ==================================================================================================
# Total params: 15,378,504
# Trainable params: 15,378,504
# Non-trainable params: 0
# __________________________________________________________________________________________________



In [ ]:
# Execute #1:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 4s 14ms/step
#  - fnc_score: 0.5901639344262295
#  - val_f1_(macro): 0.41010925133555365


# 8/8 [==============================] - 32s 1s/step - loss: 1.3160 - accuracy: 0.3369 - val_loss: 1.1453 - val_accuracy: 0.5750
# Epoch 2/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.4139344262295082
#  - val_f1_(macro): 0.3267886656162103


# 8/8 [==============================] - 2s 297ms/step - loss: 1.1080 - accuracy: 0.5642 - val_loss: 1.2043 - val_accuracy: 0.3350
# Epoch 3/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.4713114754098361
#  - val_f1_(macro): 0.4058401504462841


# 8/8 [==============================] - 2s 292ms/step - loss: 0.7850 - accuracy: 0.6237 - val_loss: 1.1669 - val_accuracy: 0.3825
# Epoch 4/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.6789617486338798
#  - val_f1_(macro): 0.58424138610627


# 8/8 [==============================] - 2s 297ms/step - loss: 0.5110 - accuracy: 0.7714 - val_loss: 0.9859 - val_accuracy: 0.6325
# Epoch 5/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7213114754098361
#  - val_f1_(macro): 0.612211311675895


# 8/8 [==============================] - 2s 235ms/step - loss: 0.2643 - accuracy: 0.8698 - val_loss: 0.9599 - val_accuracy: 0.6775
# Epoch 6/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7390710382513661
#  - val_f1_(macro): 0.6169221373160112


# 8/8 [==============================] - 3s 346ms/step - loss: 0.1428 - accuracy: 0.9261 - val_loss: 1.0874 - val_accuracy: 0.7100
# Epoch 7/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7213114754098361
#  - val_f1_(macro): 0.6232604845076861


# 8/8 [==============================] - 3s 393ms/step - loss: 0.1044 - accuracy: 0.9549 - val_loss: 1.1910 - val_accuracy: 0.6975
# Epoch 8/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7049180327868853
#  - val_f1_(macro): 0.5987446228831513


# 8/8 [==============================] - 2s 293ms/step - loss: 0.0683 - accuracy: 0.9643 - val_loss: 1.3655 - val_accuracy: 0.6825
# Epoch 9/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7602459016393442
#  - val_f1_(macro): 0.6350490341554171


# 8/8 [==============================] - 2s 271ms/step - loss: 0.0369 - accuracy: 0.9800 - val_loss: 1.3501 - val_accuracy: 0.7250
# Epoch 10/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7472677595628415
#  - val_f1_(macro): 0.6187263601317903


# 8/8 [==============================] - 2s 287ms/step - loss: 0.0253 - accuracy: 0.9894 - val_loss: 1.4648 - val_accuracy: 0.7050
# Epoch 11/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7465846994535519
#  - val_f1_(macro): 0.6243013023329066


# 8/8 [==============================] - 2s 274ms/step - loss: 0.0217 - accuracy: 0.9912 - val_loss: 1.4823 - val_accuracy: 0.7075
# Epoch 12/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.744535519125683
#  - val_f1_(macro): 0.6317682897198532


# 8/8 [==============================] - 3s 369ms/step - loss: 0.0160 - accuracy: 0.9931 - val_loss: 1.5702 - val_accuracy: 0.7150
# Epoch 13/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.7377049180327869
#  - val_f1_(macro): 0.6117449903824147


# 8/8 [==============================] - 2s 285ms/step - loss: 0.0140 - accuracy: 0.9931 - val_loss: 1.8120 - val_accuracy: 0.6925
# Epoch 14/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7404371584699454
#  - val_f1_(macro): 0.625897698108534


# 8/8 [==============================] - 2s 271ms/step - loss: 0.0121 - accuracy: 0.9937 - val_loss: 1.6393 - val_accuracy: 0.7100
# Epoch 15/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7411202185792349
#  - val_f1_(macro): 0.632026961374831


# 8/8 [==============================] - 2s 232ms/step - loss: 0.0158 - accuracy: 0.9931 - val_loss: 1.6819 - val_accuracy: 0.7075
# Epoch 16/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7520491803278688
#  - val_f1_(macro): 0.6384144656655376


# 8/8 [==============================] - 2s 271ms/step - loss: 0.0199 - accuracy: 0.9906 - val_loss: 1.6251 - val_accuracy: 0.7125
# Epoch 17/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7377049180327869
#  - val_f1_(macro): 0.6178854971248304


# 8/8 [==============================] - 2s 261ms/step - loss: 0.0161 - accuracy: 0.9962 - val_loss: 1.7077 - val_accuracy: 0.6975
# Epoch 18/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7486338797814208
#  - val_f1_(macro): 0.629396216848103


# 8/8 [==============================] - 2s 284ms/step - loss: 0.0191 - accuracy: 0.9944 - val_loss: 1.6223 - val_accuracy: 0.7100
# Epoch 19/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7459016393442623
#  - val_f1_(macro): 0.6293635979011065


# 8/8 [==============================] - 2s 276ms/step - loss: 0.0161 - accuracy: 0.9937 - val_loss: 1.6586 - val_accuracy: 0.7075
# Epoch 20/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7452185792349727
#  - val_f1_(macro): 0.6234233228764479


# 8/8 [==============================] - 2s 259ms/step - loss: 0.0096 - accuracy: 0.9956 - val_loss: 1.6102 - val_accuracy: 0.7100
# Epoch 21/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7465846994535519
#  - val_f1_(macro): 0.6167444189796052


# 8/8 [==============================] - 2s 207ms/step - loss: 0.0078 - accuracy: 0.9956 - val_loss: 1.7031 - val_accuracy: 0.7075
# CPU times: user 54.7 s, sys: 2.43 s, total: 57.1 s
# Wall time: 1min 16s


# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |     8     |     0     |    17     |     2     |
# -------------------------------------------------------------
# | Disagree  |     2     |    24     |    16     |     0     |
# -------------------------------------------------------------
# |  Discuss  |     9     |     5     |    163    |    18     |
# -------------------------------------------------------------
# | Unrelated |     1     |     2     |    45     |    88     |
# -------------------------------------------------------------


#               precision    recall  f1-score   support

#        Agree       0.40      0.30      0.34        27
#     Disagree       0.77      0.57      0.66        42
#      Discuss       0.68      0.84      0.75       195
#    Unrelated       0.81      0.65      0.72       136

#     accuracy                           0.71       400
#    macro avg       0.67      0.59      0.62       400
# weighted avg       0.72      0.71      0.70       400



In [ ]:
# Execute #2:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 3s 10ms/step
#  - fnc_score: 0.5116120218579235
#  - val_f1_(macro): 0.34279835630149014


# 8/8 [==============================] - 37s 1s/step - loss: 1.3362 - accuracy: 0.3187 - val_loss: 1.2254 - val_accuracy: 0.4725
# Epoch 2/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.5307377049180327
#  - val_f1_(macro): 0.45924889466728314


# 8/8 [==============================] - 2s 273ms/step - loss: 1.1439 - accuracy: 0.4890 - val_loss: 1.1230 - val_accuracy: 0.4875
# Epoch 3/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.6967213114754098
#  - val_f1_(macro): 0.5893514446361967


# 8/8 [==============================] - 2s 267ms/step - loss: 0.8251 - accuracy: 0.6875 - val_loss: 0.8660 - val_accuracy: 0.6650
# Epoch 4/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7219945355191257
#  - val_f1_(macro): 0.5886093300803998


# 8/8 [==============================] - 2s 298ms/step - loss: 0.4697 - accuracy: 0.7990 - val_loss: 0.8382 - val_accuracy: 0.6750
# Epoch 5/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.6666666666666666
#  - val_f1_(macro): 0.5652056146999281


# 8/8 [==============================] - 3s 355ms/step - loss: 0.2715 - accuracy: 0.8591 - val_loss: 1.2015 - val_accuracy: 0.6100
# Epoch 6/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7226775956284153
#  - val_f1_(macro): 0.6233611416717051


# 8/8 [==============================] - 3s 400ms/step - loss: 0.2006 - accuracy: 0.9092 - val_loss: 0.9775 - val_accuracy: 0.6900
# Epoch 7/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7493169398907104
#  - val_f1_(macro): 0.6420885611563577


# 8/8 [==============================] - 2s 265ms/step - loss: 0.1259 - accuracy: 0.9443 - val_loss: 0.9860 - val_accuracy: 0.7150
# Epoch 8/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.744535519125683
#  - val_f1_(macro): 0.618349327340642


# 8/8 [==============================] - 2s 294ms/step - loss: 0.0699 - accuracy: 0.9668 - val_loss: 1.2085 - val_accuracy: 0.7025
# Epoch 9/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7363387978142076
#  - val_f1_(macro): 0.627549722714585


# 8/8 [==============================] - 2s 244ms/step - loss: 0.0406 - accuracy: 0.9825 - val_loss: 1.2191 - val_accuracy: 0.7000
# Epoch 10/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7493169398907104
#  - val_f1_(macro): 0.6312241212321148


# 8/8 [==============================] - 2s 211ms/step - loss: 0.0354 - accuracy: 0.9825 - val_loss: 1.3114 - val_accuracy: 0.7100
# Epoch 11/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7363387978142076
#  - val_f1_(macro): 0.6399161305391365


# 8/8 [==============================] - 2s 226ms/step - loss: 0.0384 - accuracy: 0.9762 - val_loss: 1.4453 - val_accuracy: 0.7100
# Epoch 12/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7527322404371585
#  - val_f1_(macro): 0.6464555679661665


# 8/8 [==============================] - 2s 315ms/step - loss: 0.0246 - accuracy: 0.9831 - val_loss: 1.4730 - val_accuracy: 0.7250
# Epoch 13/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7370218579234973
#  - val_f1_(macro): 0.6298466347444309


# 8/8 [==============================] - 2s 334ms/step - loss: 0.0197 - accuracy: 0.9912 - val_loss: 1.4415 - val_accuracy: 0.7025
# Epoch 14/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7363387978142076
#  - val_f1_(macro): 0.6300121747235374


# 8/8 [==============================] - 2s 294ms/step - loss: 0.0202 - accuracy: 0.9931 - val_loss: 1.5705 - val_accuracy: 0.7100
# Epoch 15/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7418032786885246
#  - val_f1_(macro): 0.6142606107761963


# 8/8 [==============================] - 1s 205ms/step - loss: 0.0133 - accuracy: 0.9956 - val_loss: 1.7857 - val_accuracy: 0.6925
# Epoch 16/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7506830601092896
#  - val_f1_(macro): 0.6444681560444172


# 8/8 [==============================] - 2s 235ms/step - loss: 0.0117 - accuracy: 0.9962 - val_loss: 1.6149 - val_accuracy: 0.7175
# Epoch 17/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.7363387978142076
#  - val_f1_(macro): 0.6318618095852138


# 8/8 [==============================] - 2s 205ms/step - loss: 0.0114 - accuracy: 0.9962 - val_loss: 1.6338 - val_accuracy: 0.7000
# Epoch 18/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7657103825136612
#  - val_f1_(macro): 0.6473853336138156


# 8/8 [==============================] - 2s 265ms/step - loss: 0.0073 - accuracy: 0.9975 - val_loss: 1.7251 - val_accuracy: 0.7300
# Epoch 19/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7472677595628415
#  - val_f1_(macro): 0.6305192694541363


# 8/8 [==============================] - 3s 345ms/step - loss: 0.0057 - accuracy: 0.9975 - val_loss: 1.6256 - val_accuracy: 0.7125
# Epoch 20/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.7520491803278688
#  - val_f1_(macro): 0.6386343002584407


# 8/8 [==============================] - 1s 199ms/step - loss: 0.0044 - accuracy: 0.9969 - val_loss: 1.7290 - val_accuracy: 0.7200
# Epoch 21/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7629781420765027
#  - val_f1_(macro): 0.6479979415315149


# 8/8 [==============================] - 2s 227ms/step - loss: 0.0040 - accuracy: 0.9994 - val_loss: 1.7317 - val_accuracy: 0.7275
# Epoch 22/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7472677595628415
#  - val_f1_(macro): 0.6321142866795041


# 8/8 [==============================] - 2s 209ms/step - loss: 0.0052 - accuracy: 0.9962 - val_loss: 1.7718 - val_accuracy: 0.7125
# Epoch 23/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.7383879781420765
#  - val_f1_(macro): 0.629784009094354


# 8/8 [==============================] - 2s 229ms/step - loss: 0.0068 - accuracy: 0.9987 - val_loss: 1.6710 - val_accuracy: 0.7050
# CPU times: user 57.2 s, sys: 2.77 s, total: 60 s
# Wall time: 1min 23s


# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |    10     |     0     |    14     |     3     |
# -------------------------------------------------------------
# | Disagree  |     2     |    27     |    13     |     0     |
# -------------------------------------------------------------
# |  Discuss  |    13     |     6     |    150    |    26     |
# -------------------------------------------------------------
# | Unrelated |     4     |     3     |    34     |    95     |
# -------------------------------------------------------------


#               precision    recall  f1-score   support

#        Agree       0.34      0.37      0.36        27
#     Disagree       0.75      0.64      0.69        42
#      Discuss       0.71      0.77      0.74       195
#    Unrelated       0.77      0.70      0.73       136

#     accuracy                           0.70       400
#    macro avg       0.64      0.62      0.63       400
# weighted avg       0.71      0.70      0.71       400



In [ ]:
# Execute #3:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 3s 13ms/step
#  - fnc_score: 0.48292349726775957
#  - val_f1_(macro): 0.3667977855477856


# 8/8 [==============================] - 32s 1s/step - loss: 1.3601 - accuracy: 0.2893 - val_loss: 1.2483 - val_accuracy: 0.4450
# Epoch 2/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.5075136612021858
#  - val_f1_(macro): 0.44719756182042714


# 8/8 [==============================] - 3s 340ms/step - loss: 1.1811 - accuracy: 0.5241 - val_loss: 1.1505 - val_accuracy: 0.4475
# Epoch 3/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.5635245901639344
#  - val_f1_(macro): 0.5253392901115137


# 8/8 [==============================] - 2s 303ms/step - loss: 0.8831 - accuracy: 0.6518 - val_loss: 1.0483 - val_accuracy: 0.5000
# Epoch 4/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.6892076502732241
#  - val_f1_(macro): 0.6205762515630262


# 8/8 [==============================] - 2s 270ms/step - loss: 0.5246 - accuracy: 0.7771 - val_loss: 0.8055 - val_accuracy: 0.6800
# Epoch 5/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7315573770491803
#  - val_f1_(macro): 0.6134240281478445


# 8/8 [==============================] - 2s 269ms/step - loss: 0.2821 - accuracy: 0.8510 - val_loss: 0.9543 - val_accuracy: 0.6800
# Epoch 6/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7260928961748634
#  - val_f1_(macro): 0.6097943453270873


# 8/8 [==============================] - 3s 362ms/step - loss: 0.1758 - accuracy: 0.9011 - val_loss: 1.0219 - val_accuracy: 0.6875
# Epoch 7/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7083333333333334
#  - val_f1_(macro): 0.592731788525699


# 8/8 [==============================] - 3s 331ms/step - loss: 0.1162 - accuracy: 0.9443 - val_loss: 1.1817 - val_accuracy: 0.6675
# Epoch 8/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.75
#  - val_f1_(macro): 0.6369719722229542


# 8/8 [==============================] - 2s 298ms/step - loss: 0.0820 - accuracy: 0.9606 - val_loss: 1.1536 - val_accuracy: 0.7150
# Epoch 9/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7561475409836066
#  - val_f1_(macro): 0.6240393930534776


# 8/8 [==============================] - 2s 264ms/step - loss: 0.0636 - accuracy: 0.9681 - val_loss: 1.2713 - val_accuracy: 0.7175
# Epoch 10/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7370218579234973
#  - val_f1_(macro): 0.6018095052766229


# 8/8 [==============================] - 2s 208ms/step - loss: 0.0439 - accuracy: 0.9818 - val_loss: 1.3965 - val_accuracy: 0.6975
# Epoch 11/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.7117486338797814
#  - val_f1_(macro): 0.6056618444455247


# 8/8 [==============================] - 2s 232ms/step - loss: 0.0327 - accuracy: 0.9856 - val_loss: 1.3678 - val_accuracy: 0.6825
# Epoch 12/70
# 13/13 [==============================] - 0s 13ms/step
#  - fnc_score: 0.7199453551912568
#  - val_f1_(macro): 0.619587172587806


# 8/8 [==============================] - 2s 295ms/step - loss: 0.0242 - accuracy: 0.9900 - val_loss: 1.3895 - val_accuracy: 0.6875
# Epoch 13/70
# 13/13 [==============================] - 0s 14ms/step
#  - fnc_score: 0.755464480874317
#  - val_f1_(macro): 0.6347974436263027


# 8/8 [==============================] - 3s 363ms/step - loss: 0.0234 - accuracy: 0.9862 - val_loss: 1.5526 - val_accuracy: 0.7200
# Epoch 14/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7540983606557377
#  - val_f1_(macro): 0.634611282400311


# 8/8 [==============================] - 2s 215ms/step - loss: 0.0134 - accuracy: 0.9937 - val_loss: 1.4965 - val_accuracy: 0.7225
# Epoch 15/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.75
#  - val_f1_(macro): 0.6373465665609902


# 8/8 [==============================] - 2s 236ms/step - loss: 0.0090 - accuracy: 0.9956 - val_loss: 1.5721 - val_accuracy: 0.7175
# Epoch 16/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7527322404371585
#  - val_f1_(macro): 0.623712352700541


# 8/8 [==============================] - 2s 271ms/step - loss: 0.0075 - accuracy: 0.9975 - val_loss: 1.6856 - val_accuracy: 0.7125
# Epoch 17/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7418032786885246
#  - val_f1_(macro): 0.6239780326262884


# 8/8 [==============================] - 2s 298ms/step - loss: 0.0064 - accuracy: 0.9975 - val_loss: 1.6048 - val_accuracy: 0.7075
# Epoch 18/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.7486338797814208
#  - val_f1_(macro): 0.6177409060149054


# 8/8 [==============================] - 2s 233ms/step - loss: 0.0065 - accuracy: 0.9975 - val_loss: 1.7047 - val_accuracy: 0.7150
# Epoch 19/70
# 13/13 [==============================] - 0s 13ms/step
#  - fnc_score: 0.7452185792349727
#  - val_f1_(macro): 0.6266717031511204


# 8/8 [==============================] - 3s 383ms/step - loss: 0.0082 - accuracy: 0.9975 - val_loss: 1.6503 - val_accuracy: 0.7100
# Epoch 20/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7452185792349727
#  - val_f1_(macro): 0.6218991306287927


# 8/8 [==============================] - 2s 182ms/step - loss: 0.0069 - accuracy: 0.9975 - val_loss: 1.6715 - val_accuracy: 0.7100
# Epoch 21/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.75
#  - val_f1_(macro): 0.6317748491000771


# 8/8 [==============================] - 2s 182ms/step - loss: 0.0070 - accuracy: 0.9975 - val_loss: 1.7035 - val_accuracy: 0.7175
# Epoch 22/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7581967213114754
#  - val_f1_(macro): 0.638662547298342


# 8/8 [==============================] - 2s 229ms/step - loss: 0.0045 - accuracy: 0.9987 - val_loss: 1.7392 - val_accuracy: 0.7250
# Epoch 23/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.7274590163934426
#  - val_f1_(macro): 0.6267305991043987


# 8/8 [==============================] - 1s 200ms/step - loss: 0.0063 - accuracy: 0.9981 - val_loss: 1.6584 - val_accuracy: 0.6975
# Epoch 24/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7657103825136612
#  - val_f1_(macro): 0.6259836674090571


# 8/8 [==============================] - 1s 176ms/step - loss: 0.0071 - accuracy: 0.9969 - val_loss: 1.8927 - val_accuracy: 0.7275
# CPU times: user 58.4 s, sys: 2.65 s, total: 1min 1s
# Wall time: 1min 21s

# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |     7     |     0     |    18     |     2     |
# -------------------------------------------------------------
# | Disagree  |     2     |    23     |    17     |     0     |
# -------------------------------------------------------------
# |  Discuss  |     5     |     3     |    173    |    14     |
# -------------------------------------------------------------
# | Unrelated |     1     |     1     |    46     |    88     |
# -------------------------------------------------------------


#               precision    recall  f1-score   support

#        Agree       0.47      0.26      0.33        27
#     Disagree       0.85      0.55      0.67        42
#      Discuss       0.68      0.89      0.77       195
#    Unrelated       0.85      0.65      0.73       136

#     accuracy                           0.73       400
#    macro avg       0.71      0.59      0.63       400
# weighted avg       0.74      0.73      0.72       400

In [ ]:
# Execute #4:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 2s 11ms/step
#  - fnc_score: 0.375
#  - val_f1_(macro): 0.24965915484685852


# 8/8 [==============================] - 38s 1s/step - loss: 1.3580 - accuracy: 0.3106 - val_loss: 1.3858 - val_accuracy: 0.2625
# Epoch 2/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.5792349726775956
#  - val_f1_(macro): 0.47912982174762364


# 8/8 [==============================] - 2s 318ms/step - loss: 1.1942 - accuracy: 0.4590 - val_loss: 1.1134 - val_accuracy: 0.5600
# Epoch 3/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.6994535519125683
#  - val_f1_(macro): 0.6090879161548004


# 8/8 [==============================] - 3s 325ms/step - loss: 0.8714 - accuracy: 0.6775 - val_loss: 0.8717 - val_accuracy: 0.6700
# Epoch 4/70
# 13/13 [==============================] - 0s 13ms/step
#  - fnc_score: 0.7575136612021858
#  - val_f1_(macro): 0.64240412721704


# 8/8 [==============================] - 3s 333ms/step - loss: 0.4739 - accuracy: 0.7833 - val_loss: 0.7904 - val_accuracy: 0.7375
# Epoch 5/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7308743169398907
#  - val_f1_(macro): 0.5932031328048455


# 8/8 [==============================] - 3s 422ms/step - loss: 0.2915 - accuracy: 0.8547 - val_loss: 0.9353 - val_accuracy: 0.6825
# Epoch 6/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7486338797814208
#  - val_f1_(macro): 0.6258308550658329


# 8/8 [==============================] - 2s 274ms/step - loss: 0.1618 - accuracy: 0.9192 - val_loss: 1.0058 - val_accuracy: 0.7075
# Epoch 7/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7459016393442623
#  - val_f1_(macro): 0.6306744801154051


# 8/8 [==============================] - 2s 246ms/step - loss: 0.0918 - accuracy: 0.9537 - val_loss: 1.0966 - val_accuracy: 0.7025
# Epoch 8/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7411202185792349
#  - val_f1_(macro): 0.6166244605752673


# 8/8 [==============================] - 2s 291ms/step - loss: 0.0707 - accuracy: 0.9706 - val_loss: 1.2453 - val_accuracy: 0.7000
# Epoch 9/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7383879781420765
#  - val_f1_(macro): 0.6143121622150309


# 8/8 [==============================] - 2s 271ms/step - loss: 0.0380 - accuracy: 0.9862 - val_loss: 1.3566 - val_accuracy: 0.6975
# Epoch 10/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7260928961748634
#  - val_f1_(macro): 0.6141184892584758


# 8/8 [==============================] - 3s 390ms/step - loss: 0.0260 - accuracy: 0.9906 - val_loss: 1.4140 - val_accuracy: 0.7000
# Epoch 11/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.755464480874317
#  - val_f1_(macro): 0.6330735737174369


# 8/8 [==============================] - 3s 325ms/step - loss: 0.0155 - accuracy: 0.9956 - val_loss: 1.4649 - val_accuracy: 0.7175
# Epoch 12/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7233606557377049
#  - val_f1_(macro): 0.6196608040201005


# 8/8 [==============================] - 2s 269ms/step - loss: 0.0190 - accuracy: 0.9944 - val_loss: 1.5242 - val_accuracy: 0.6900
# Epoch 13/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7281420765027322
#  - val_f1_(macro): 0.6294742364435957


# 8/8 [==============================] - 2s 283ms/step - loss: 0.0259 - accuracy: 0.9894 - val_loss: 1.4480 - val_accuracy: 0.6975
# Epoch 14/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7465846994535519
#  - val_f1_(macro): 0.6146574014221073


# 8/8 [==============================] - 3s 330ms/step - loss: 0.0336 - accuracy: 0.9869 - val_loss: 1.6618 - val_accuracy: 0.7050
# Epoch 15/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7520491803278688
#  - val_f1_(macro): 0.6065734733349416


# 8/8 [==============================] - 2s 281ms/step - loss: 0.0276 - accuracy: 0.9862 - val_loss: 1.6500 - val_accuracy: 0.7125
# Epoch 16/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7506830601092896
#  - val_f1_(macro): 0.6216884641180416


# 8/8 [==============================] - 3s 366ms/step - loss: 0.0309 - accuracy: 0.9900 - val_loss: 1.6077 - val_accuracy: 0.7150
# Epoch 17/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.7540983606557377
#  - val_f1_(macro): 0.6250225198204413


# 8/8 [==============================] - 1s 185ms/step - loss: 0.0278 - accuracy: 0.9869 - val_loss: 1.5065 - val_accuracy: 0.7150
# Epoch 18/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7390710382513661
#  - val_f1_(macro): 0.6234788883140072


# 8/8 [==============================] - 2s 266ms/step - loss: 0.0158 - accuracy: 0.9956 - val_loss: 1.5496 - val_accuracy: 0.7075
# Epoch 19/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7588797814207651
#  - val_f1_(macro): 0.6299100889753376


# 8/8 [==============================] - 2s 274ms/step - loss: 0.0122 - accuracy: 0.9944 - val_loss: 1.6031 - val_accuracy: 0.7225
# Epoch 20/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7459016393442623
#  - val_f1_(macro): 0.6222693479907686


# 8/8 [==============================] - 2s 276ms/step - loss: 0.0143 - accuracy: 0.9925 - val_loss: 1.5437 - val_accuracy: 0.7100
# Epoch 21/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.744535519125683
#  - val_f1_(macro): 0.6389419715121509


# 8/8 [==============================] - 2s 300ms/step - loss: 0.0115 - accuracy: 0.9962 - val_loss: 1.5108 - val_accuracy: 0.7125
# Epoch 22/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.755464480874317
#  - val_f1_(macro): 0.6332397562600633


# 8/8 [==============================] - 2s 235ms/step - loss: 0.0151 - accuracy: 0.9912 - val_loss: 1.6623 - val_accuracy: 0.7200
# Epoch 23/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7062841530054644
#  - val_f1_(macro): 0.5915427861534575


# 8/8 [==============================] - 2s 277ms/step - loss: 0.0201 - accuracy: 0.9931 - val_loss: 1.6961 - val_accuracy: 0.6600
# CPU times: user 1min, sys: 2.68 s, total: 1min 2s
# Wall time: 1min 28s


# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |    14     |     1     |     9     |     3     |
# -------------------------------------------------------------
# | Disagree  |     3     |    29     |    10     |     0     |
# -------------------------------------------------------------
# |  Discuss  |    23     |    14     |    139    |    19     |
# -------------------------------------------------------------
# | Unrelated |    17     |     7     |    30     |    82     |
# -------------------------------------------------------------


#               precision    recall  f1-score   support

#        Agree       0.25      0.52      0.33        27
#     Disagree       0.57      0.69      0.62        42
#      Discuss       0.74      0.71      0.73       195
#    Unrelated       0.79      0.60      0.68       136

#     accuracy                           0.66       400
#    macro avg       0.59      0.63      0.59       400
# weighted avg       0.70      0.66      0.67       400

In [ ]:
# Execute #5:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 2s 11ms/step
#  - fnc_score: 0.46379781420765026
#  - val_f1_(macro): 0.3366819395557713


# 8/8 [==============================] - 28s 958ms/step - loss: 1.3495 - accuracy: 0.3012 - val_loss: 1.3103 - val_accuracy: 0.3750
# Epoch 2/70
# 13/13 [==============================] - 0s 14ms/step
#  - fnc_score: 0.5676229508196722
#  - val_f1_(macro): 0.4649217776228891


# 8/8 [==============================] - 4s 464ms/step - loss: 1.1608 - accuracy: 0.4934 - val_loss: 1.1270 - val_accuracy: 0.5225
# Epoch 3/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.5635245901639344
#  - val_f1_(macro): 0.5011974585681692


# 8/8 [==============================] - 3s 300ms/step - loss: 0.8940 - accuracy: 0.6005 - val_loss: 1.0089 - val_accuracy: 0.5825
# Epoch 4/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7377049180327869
#  - val_f1_(macro): 0.6294340480837418


# 8/8 [==============================] - 3s 342ms/step - loss: 0.5555 - accuracy: 0.7470 - val_loss: 0.7913 - val_accuracy: 0.7225
# Epoch 5/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.7418032786885246
#  - val_f1_(macro): 0.629320491190843


# 8/8 [==============================] - 2s 295ms/step - loss: 0.3203 - accuracy: 0.8635 - val_loss: 0.8619 - val_accuracy: 0.7075
# Epoch 6/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7151639344262295
#  - val_f1_(macro): 0.5948271811151902


# 8/8 [==============================] - 2s 300ms/step - loss: 0.1770 - accuracy: 0.9029 - val_loss: 1.0396 - val_accuracy: 0.6700
# Epoch 7/70
# 13/13 [==============================] - 0s 13ms/step
#  - fnc_score: 0.7459016393442623
#  - val_f1_(macro): 0.6189745586708204


# 8/8 [==============================] - 3s 429ms/step - loss: 0.0976 - accuracy: 0.9480 - val_loss: 1.2058 - val_accuracy: 0.7100
# Epoch 8/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7452185792349727
#  - val_f1_(macro): 0.6183245943519916


# 8/8 [==============================] - 2s 303ms/step - loss: 0.0480 - accuracy: 0.9793 - val_loss: 1.3927 - val_accuracy: 0.7025
# Epoch 9/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7520491803278688
#  - val_f1_(macro): 0.6272468103992214


# 8/8 [==============================] - 3s 320ms/step - loss: 0.0307 - accuracy: 0.9862 - val_loss: 1.5287 - val_accuracy: 0.7125
# Epoch 10/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7411202185792349
#  - val_f1_(macro): 0.6045687749718951


# 8/8 [==============================] - 1s 188ms/step - loss: 0.0327 - accuracy: 0.9881 - val_loss: 1.6085 - val_accuracy: 0.6950
# Epoch 11/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7137978142076503
#  - val_f1_(macro): 0.6165176902187267


# 8/8 [==============================] - 2s 213ms/step - loss: 0.0220 - accuracy: 0.9900 - val_loss: 1.5479 - val_accuracy: 0.6825
# Epoch 12/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7103825136612022
#  - val_f1_(macro): 0.6173614675699437


# 8/8 [==============================] - 2s 271ms/step - loss: 0.0211 - accuracy: 0.9912 - val_loss: 1.7053 - val_accuracy: 0.6875
# Epoch 13/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7213114754098361
#  - val_f1_(macro): 0.5920676182249027


# 8/8 [==============================] - 2s 297ms/step - loss: 0.0168 - accuracy: 0.9937 - val_loss: 1.8162 - val_accuracy: 0.6750
# Epoch 14/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7424863387978142
#  - val_f1_(macro): 0.5992066152425081


# 8/8 [==============================] - 3s 368ms/step - loss: 0.0176 - accuracy: 0.9925 - val_loss: 1.9755 - val_accuracy: 0.6975
# Epoch 15/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7336065573770492
#  - val_f1_(macro): 0.6058402603014819


# 8/8 [==============================] - 2s 238ms/step - loss: 0.0151 - accuracy: 0.9950 - val_loss: 2.0740 - val_accuracy: 0.6850
# Epoch 16/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7288251366120219
#  - val_f1_(macro): 0.6278707115340778


# 8/8 [==============================] - 2s 216ms/step - loss: 0.0120 - accuracy: 0.9969 - val_loss: 1.6819 - val_accuracy: 0.7000
# Epoch 17/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7390710382513661
#  - val_f1_(macro): 0.6269375045691581


# 8/8 [==============================] - 2s 214ms/step - loss: 0.0101 - accuracy: 0.9975 - val_loss: 1.7225 - val_accuracy: 0.7025
# Epoch 18/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7452185792349727
#  - val_f1_(macro): 0.6221110683596255


# 8/8 [==============================] - 3s 344ms/step - loss: 0.0084 - accuracy: 0.9962 - val_loss: 1.9560 - val_accuracy: 0.7025
# Epoch 19/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.7192622950819673
#  - val_f1_(macro): 0.6152143925705738


# 8/8 [==============================] - 2s 222ms/step - loss: 0.0065 - accuracy: 0.9981 - val_loss: 1.7470 - val_accuracy: 0.6925
# Epoch 20/70
# 13/13 [==============================] - 0s 13ms/step
#  - fnc_score: 0.744535519125683
#  - val_f1_(macro): 0.6073840010781312


# 8/8 [==============================] - 3s 405ms/step - loss: 0.0069 - accuracy: 0.9962 - val_loss: 2.1678 - val_accuracy: 0.6975
# Epoch 21/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.730191256830601
#  - val_f1_(macro): 0.6290858786729217


# 8/8 [==============================] - 2s 265ms/step - loss: 0.0051 - accuracy: 0.9975 - val_loss: 1.8517 - val_accuracy: 0.7025
# Epoch 22/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7363387978142076
#  - val_f1_(macro): 0.6157038157728845


# 8/8 [==============================] - 2s 242ms/step - loss: 0.0029 - accuracy: 0.9987 - val_loss: 1.9964 - val_accuracy: 0.6950
# Epoch 23/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.7459016393442623
#  - val_f1_(macro): 0.6291829177307061


# 8/8 [==============================] - 2s 209ms/step - loss: 0.0029 - accuracy: 0.9981 - val_loss: 1.9448 - val_accuracy: 0.7075
# Epoch 24/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.735655737704918
#  - val_f1_(macro): 0.6159303185966869


# 8/8 [==============================] - 2s 206ms/step - loss: 0.0028 - accuracy: 0.9987 - val_loss: 2.0958 - val_accuracy: 0.6950
# Epoch 25/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.7397540983606558
#  - val_f1_(macro): 0.6221989038836615


# 8/8 [==============================] - 1s 114ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 1.9829 - val_accuracy: 0.7025
# Epoch 26/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.7383879781420765
#  - val_f1_(macro): 0.6242902653052067


# 8/8 [==============================] - 1s 147ms/step - loss: 0.0033 - accuracy: 0.9987 - val_loss: 2.1423 - val_accuracy: 0.6975
# Epoch 27/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.75
#  - val_f1_(macro): 0.6370225301875897


# 8/8 [==============================] - 2s 265ms/step - loss: 0.0038 - accuracy: 0.9987 - val_loss: 2.0059 - val_accuracy: 0.7125
# Epoch 28/70
# 13/13 [==============================] - 0s 13ms/step
#  - fnc_score: 0.7472677595628415
#  - val_f1_(macro): 0.6307902780838032


# 8/8 [==============================] - 2s 316ms/step - loss: 0.0033 - accuracy: 0.9987 - val_loss: 2.0183 - val_accuracy: 0.7100
# Epoch 29/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.7438524590163934
#  - val_f1_(macro): 0.6270440584472928


# 8/8 [==============================] - 1s 181ms/step - loss: 0.0030 - accuracy: 0.9994 - val_loss: 2.0709 - val_accuracy: 0.7050
# CPU times: user 1min 2s, sys: 3.01 s, total: 1min 5s
# Wall time: 1min 27s


# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |    10     |     0     |    15     |     2     |
# -------------------------------------------------------------
# | Disagree  |     2     |    24     |    16     |     0     |
# -------------------------------------------------------------
# |  Discuss  |    10     |     5     |    161    |    19     |
# -------------------------------------------------------------
# | Unrelated |     2     |     2     |    45     |    87     |
# -------------------------------------------------------------



#               precision    recall  f1-score   support

#        Agree       0.42      0.37      0.39        27
#     Disagree       0.77      0.57      0.66        42
#      Discuss       0.68      0.83      0.75       195
#    Unrelated       0.81      0.64      0.71       136

#     accuracy                           0.70       400
#    macro avg       0.67      0.60      0.63       400
# weighted avg       0.71      0.70      0.70       400
